<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/HighVolatilityStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
!pip install pandas-ta

!pip install TA-Lib

import pandas as pd
import yfinance as yf
import pandas_ta as ta
import requests

# List of assets to monitor
assets = ['AAPL', 'GOOGL', 'TSLA', 'AMZN', '^XAU', '^GDAXI', '^GSPC']

# Define criteria thresholds
MAX_RSI = 70  # Overbought condition
MIN_RSI = 30  # Oversold condition
MAX_VOLATILITY = 2  # Arbitrary threshold for volatility
MIN_VOLUME = 1000000  # Minimum trading volume

# News API configuration
NEWS_API_KEY = 'your_news_api_key'  # Replace with your NewsAPI key
NEWS_API_URL = 'https://newsapi.org/v2/everything'

# Fetch data
def get_asset_data(asset):
    data = yf.download(asset, period='1mo', interval='1d')
    return data

# Fetch news
def fetch_news(asset):
    params = {
        'q': asset,
        'apiKey': NEWS_API_KEY,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': 5  # Get top 5 news articles
    }
    response = requests.get(NEWS_API_URL, params=params)
    news_data = response.json()

    if news_data['status'] == 'ok':
        return news_data['articles']
    else:
        return []

# Analyze data
def analyze_data(asset):
    data = get_asset_data(asset)

    if data.empty:
        return {
            'asset': asset,
            'rsi': None,
            'volatility': None,
            'volume': None,
            'error': 'No data available',
            'news': None
        }

    # Calculate RSI using pandas-ta
    data['RSI'] = ta.rsi(data['Close'], length=14)

    # Calculate Bollinger Bands
    bb = ta.bbands(data['Close'], length=20, std=2)
    data['BB_Width'] = bb['BBU_20_2.0'] - bb['BBL_20_2.0']

    # Check volume (applicable mainly to stocks, not indexes or commodities)
    latest_volume = data['Volume'].iloc[-1] if 'Volume' in data.columns else None
    latest_rsi = data['RSI'].iloc[-1]
    latest_volatility = data['BB_Width'].iloc[-1]

    # Fetch latest news
    news = fetch_news(asset)

    return {
        'asset': asset,
        'rsi': latest_rsi,
        'volatility': latest_volatility,
        'volume': latest_volume,
        'news': news
    }

# Check assets
def check_assets(assets):
    alerts = []
    for asset in assets:
        result = analyze_data(asset)
        if result.get('error'):
            alerts.append(f"Alert for {asset}: {result['error']}.")
            continue
        if result['rsi'] > MAX_RSI or result['rsi'] < MIN_RSI:
            alerts.append(f"Alert for {asset}: RSI indicates overbought/oversold conditions.")
        if result['volatility'] > MAX_VOLATILITY:
            alerts.append(f"Alert for {asset}: High volatility detected.")
        if result['volume'] is not None and result['volume'] < MIN_VOLUME:
            alerts.append(f"Alert for {asset}: Low trading volume detected.")

        # Check for negative news
        if result['news']:
            for article in result['news']:
                if "risk" in article['title'].lower() or "warning" in article['title'].lower():
                    alerts.append(f"Alert for {asset}: Negative news detected - {article['title']}")

    return alerts

# Get alerts
alerts = check_assets(assets)

# Display alerts
if alerts:
    for alert in alerts:
        print(alert)
else:
    print("No alerts. All assets are good to trade today.")


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=5464ca905a7bb785914a204d28ad8efb409ba77ee227c249d0208403d14ea532
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=687080bcca58c4556100dc031f14a1cb7d3afaab1a665fc83d46b5fe05acad6a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Buildi

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Alert for AAPL: High volatility detected.
Alert for GOOGL: High volatility detected.
Alert for TSLA: High volatility detected.
Alert for AMZN: High volatility detected.
Alert for ^XAU: High volatility detected.
Alert for ^XAU: Low trading volume detected.
Alert for ^GDAXI: RSI indicates overbought/oversold conditions.
Alert for ^GDAXI: High volatility detected.
Alert for ^GSPC: High volatility detected.
